<a href="https://colab.research.google.com/github/Guhabrus/Laba_1_CUDA/blob/main/Laba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-olf_k2j2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-olf_k2j2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=0edc49a954e6901d16d9b24aaec4a0f9c1a86d22dd2b7b79fda95ab5bcf6ca93
  Stored in directory: /tmp/pip-ephem-wheel-cache-a6p626xc/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [16]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include "device_functions.h"
#include <stdio.h>
#include <fstream>
//#include "../common/cpu_bitmap.h"





const int Count = 65532;                           //кол-во выводимых чисел

__global__ void printMessagdeGPU(int *a)          //Функция выполняема на GPU
{
    int Value = threadIdx.x + blockIdx.x * blockDim.x ;
    if(Value <= Count)
    {
        printf("ValueGPU = %d;\n", a[Value]);
    }
    
}

void printMessageCPU()                      //Функция выполняемая на CPU
{
    for(int i=0;i<Count; i++)
    {
        printf("ValueCPU = %d;\n", i);
    }
}


int main(void)
{


    int a[Count];
    int *dev_a;
 
    cudaMalloc((void**)&dev_a, Count*sizeof(int));
 
    cudaMemcpy(dev_a, a , Count*sizeof(int), cudaMemcpyHostToDevice);

	cudaEvent_t start, stop;
	float gpu_time = 0.0;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
 
    printMessagdeGPU<<< (Count+1023)/1024 , 1024 >>>(dev_a);
 
    cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	
	cudaEventDestroy(start);
	cudaEventDestroy(stop);
    printf("time = %2fmiliseconds;",gpu_time);
   
    cudaFree(dev_a);


	return 0;
}

time = 0.004000miliseconds;
